
<header style="padding:1px;background:#00233c;border-top:5px solid #00b2b1">

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>teradataml Data Transfer Utilities - Data Loading APIs </b>
</header>

<header style="padding:1px;border-top:3px solid #00b2b1">
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Disclaimer </b> </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.
</p>

<header style="padding:1px;border-top:3px solid #00b2b1">

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
This demo notebook introduces <font color=#E37C4D>data loading APIs</font> provided by <font color=#E37C4D>teradataml</font> designed to simplify and streamline the movement of data between various sources and Teradata Vantage as destination.
Through a series of examples, the notebook showcases functions and methods that handle tasks such as bulk data loading, CSV imports, with a focus on leveraging performance features like <font color=#E37C4D>fastload protocol</font>.
Whether dealing with small datasets or large volumes of data, <font color=#E37C4D>teradataml</font> offers versatile APIs to manage data loading efficiently and reliably.
</p>


<header style="padding:1px;border-top:3px solid #00b2b1">
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Why teradataml?</b></p>  
<ul style = 'font-size:16px;color:#00233C'>
    <li>Syntax similar to opensource packages like pandas.</li>
    <li>Enable usage of Teradata specific performance improving features like fastload, from python client.</li>
    <li>Flexibility to choose number of fastload sessions to be opened for given fastload operation.</li>
    <li>Flexibility to choose batch size for loading.</li>
    <li>Easy way of monitoring erroneous records which are not loaded.</li>
 </ul>

<header style="padding:1px;border-top:3px solid #00b2b1">
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Overview of data loading APIs in teradataml</b>

| <span style=color:#E37C4D>API name</span>           | Source                                | Destination       | Function/Method | Fastload applicable | Recommended when | Returns                                                                                             |
--------------------|---------------------------------------|-------------------|-----------------|---------------------|-------------------------------------------------------------------------------------------------------------|------------|
| <span style=color:#E37C4D>copy_to_sql()</span>      | Pandas DataFrame/ terdataml DataFrame | Teradata Vantage | Function        | No                  | Number of rows < 100,000| None|                                                                                |
|<span style=color:#E37C4D>fastload()</span>         | Pandas DataFrame | Teradata Vantage | Function        | Yes                 | Number of rows > 100,000| A dictionary containing keys named as errors_dataframe, warnings_dataframe, errors_table, warnings_table, ERR_1_table, ERR_2_table    |                                                                      
|<span style=color:#E37C4D>read_csv()</span>         | csv file                              | Teradata Vantage | Function        | Yes                 | with fastload=True when Number of rows > 100,000 and with fastload=False when Number of rows < 100,000 | when fastload=False: teradataml DataFrame and when fastload=True: a tuple containing teradataml DataFrame and a dictionary. Dictionary contains keys named as errors_dataframe, warnings_dataframe, errors_table, warnings_table, fastloadcsv_error_tables |
|<span style=color:#E37C4D>DataFrame.to_sql()</span> | terdataml DataFrame                   | Teradata Vantage | Method of DataFrame class         | No                  | NA        | None|


<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Notes</b>

<style>
  li {
    margin-bottom: 5px;
  }
</style>

<ul style = 'font-size:16px;color:#00233C'>
  <li>User must have required privileges to create new tables.</li>
  <li>User should precisely specify information about column datatypes while creating new tables.</li>
  <li>Every API is having some default behavior. Teradata recommends to use <b>help()</b> on APIs before using them. 
  <li>Limitations and points to be considered while using fastload protocol are explained in below section in detail.</li>
 </ul>

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>About Fastload Protocol</b>

<style>
  li {
    margin-bottom: 5px;
  }
</style>

<ul style = 'font-size:16px;color:#00233C'>
    <li>Usage recommendation</li>
        Teradata recommends to use Fastload when number of rows in dataset is atleast 100,000. To load lesser number of rows, go with <b>copy_to_sql()</b>, <b>read_csv()</b> with <b>use_fastload=False</b>. Fastload opens multiple data transfer connections to the Teradata Vantage, which is time consuming.
    <br>
    <br>
    <li>Limitations</li>
     <ul>
      <li>FastLoad cannot load duplicate rows into a MULTISET table with a primary index.</li>
      <li>FastLoad does not support all Teradata Vantage data types. For example, BLOB and CLOB are not supported.</li>
     <ul>
</ul>

<ul style = 'font-size:16px;color:#00233C'>
<li>Find more deatils about Fastload protocol here: <a href="https://pypi.org/project/teradatasql/#FastLoad" target="_blank">https://pypi.org/project/teradatasql/#FastLoad</a></li>
</ul>

<hr>

<header style="padding:1px;border-top:3px solid #00b2b1">
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>Data loading APIs in teradataml</b>

<hr>
<b style = 'font-size:18px;font-family:Arial;color:#E37C4D'>1. Common Initial Setup</b>

#### 1.1 Import required teradataml modules and other dependencies

In [1]:
from teradataml import create_context, remove_context, DataFrame, load_example_data, db_drop_table, execute_sql, in_schema
from teradataml import copy_to_sql, fastload, read_csv
import pandas as pd
import getpass, os, json, csv
from collections import OrderedDict
from teradatasqlalchemy.types import *

#### 1.2. Connect to a Vantage system

In [2]:
# Create a connetion.
host = getpass.getpass("Host: ")
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")

create_context(host=host, username=username, password=password)

Host:  ········
Username:  ········
Password:  ········


Engine(teradatasql://user:***host)

#### 1.3 Create demo data

##### 1.3.1 Pandas Dataframe

In [3]:
data_dict = {'emp_name': ['A1', 'A2', 'A3', 'A4'],
             'emp_sage': [100, 200, 300, 400],
             'emp_id': [133, 144, 155, 177],
             'marks': [99.99, 97.32, 94.67, 91.00]
            }

pandas_df = pd.DataFrame(data_dict)

##### 1.3.2 teradataml DataFrame

In [4]:
load_example_data('dataframe', 'admissions_train')
tdml_df = DataFrame('admissions_train')

##### 1.3.3 teradataml DataFrame having timestamp data

In [5]:
load_example_data('sessionize', 'sessionize_table')
tdml_time_df = DataFrame('sessionize_table')

<hr>
<b style = 'font-size:18px;font-family:Arial;color:#E37C4D'>2. Demonstration of Data loading APIs in teradataml</b>

<hr>
<b style = 'font-size:17px;font-family:Arial;color:#E37C4D'>2.1 copy_to_sql</b>

##### Notes:
- If destination table already exists, source data will be appended to existing table by default. Use `if_exists` argument to alter this behavior.
- Destination table is created as MULTISET table by default. Use `set_table` argument to alter this behavior.
- Index columns in source pandas DataFrame will not be reflected in destination table by default. Use `index`, `index_label` arguments to alter this behavior.
- Destination table is created as a permanent table by default. Use `temporary` argument to alter this behavior.
- Destination table is created in default database. Use `schema_name` argument to alter this behavior.
- `schema_name` will be ignored when `temporary=True`.
- `primary_time_index_name`, `timecode_column`, `timebucket_duration`, `timezero_date`, `columns_list`, `sequence_column`, `seq_max` are only applicable for time series data.
- Data will be loaded in chunks of default size. Use `chunksize` argument to alter this size.

<hr>
<b style = 'font-size:16px;font-family:Arial;color:#E37C4D'>2.1.1 copy_to_sql : Source- pandas DataFrame, Destination- Teradata Vantage</b>

In [6]:
# Print source pandas DataFrame.
pandas_df

,emp_name,emp_sage,emp_id,marks
0,A1,100,133,99.99
1,A2,200,144,97.32
2,A3,300,155,94.67
3,A4,400,177,91.00


In [7]:
# Print index of pandas DataFrame.
pandas_df.index 

RangeIndex(start=0, stop=4, step=1)

#### Usecase 1: Save Pandas DataFrame to Teradata Vantage by only specifying destination table name.

Note:
- If table already exists, new data is appended to existing table. If table does not exist, `copy_to_sql()` creates table and loads data.

In [12]:
execute_sql("drop table copy_to_pd_table;")

TeradataCursor uRowsHandle=72 bClosed=False

In [13]:
copy_to_sql(df=pandas_df,
            table_name='copy_to_pd_table')

In [14]:
tdml_df_1 = DataFrame('copy_to_pd_table')
tdml_df_1

emp_name,emp_sage,emp_id,marks
A3,300,155,94.67
A4,400,177,91.0
A2,200,144,97.32
A1,100,133,99.99


In [15]:
# Default types are assigned by copy_to_sql().
tdml_df_1.tdtypes

COLUMN NAME,TYPE
emp_name,"VARCHAR(length=1024, charset='UNICODE')"
emp_sage,BIGINT()
emp_id,BIGINT()
marks,FLOAT()


#### Usecase 2: Save Pandas DataFrame to Teradata Vantage in a fresh table. Also, specify `types` argument to change default column types."

Notes:
- User can provide column names and their types in dictionary. Specify column name as key and column types as value in dictionary. Type should be a valid teradatasqlalachemy datatype.
- If `types` argument is not provided, default types are extracted from pandas dataframe and used.
- Consider indices in pandas dataframe while stating column names.

In [16]:
copy_to_sql(df=pandas_df,
            table_name='copy_to_pd_table_1',
            if_exists='replace',
            types={'emp_name': VARCHAR, 'emp_sage':INTEGER,
                   'emp_id': BIGINT, 'marks': DECIMAL})

In [17]:
tdml_df_2 = DataFrame('copy_to_pd_table_1')

In [18]:
# User defined types are assigned by copy_to_sql().
tdml_df_2.tdtypes

COLUMN NAME,TYPE
emp_name,"VARCHAR(length=64000, charset='LATIN')"
emp_sage,INTEGER()
emp_id,BIGINT()
marks,DECIMAL()


#### Usecase 3: Save Pandas DataFrame to Teradata Vantage as a SET table. Also, specify column to be used as primary index for destination table.

In [19]:
copy_to_sql(df=pandas_df,
            table_name='copy_to_pd_set_table',
            set_table=True,
            primary_index=['emp_id'])

In [20]:
set_df = DataFrame('copy_to_pd_set_table')

In [21]:
set_df.index

['emp_id']

#### Usecase 4: Save Pandas DataFrame with index to Teradata Vantage table and retain index column in destination table.

##### Notes:
- If `index` is **True** and `index_label` is not specified, then the `names` property of the Pandas DataFrame Index is used as the `index_label`(s). If pandas DataFrame index `names` is also None or empty, then:
  - A default label `index_label` or `level_0` (when `index_label` is already taken) is used when Pandas DataFrame index is standard.
  - Default labels `level_0`, `level_1`, etc. are used when Pandas DataFrame index is multi-level index.
- Use `index_label=True` only when source is Pandas DataFrame (and not when source is teradataml DataFrame).
- `index` and `index_label` arguments do not play any role in setting primary index to destination table.

In [22]:
copy_to_sql(df=pandas_df,
            table_name='copy_to_pd_table_2',
            index=True)

In [24]:
# Default label for index is used.
tdml_df_with_index_col = DataFrame('copy_to_pd_table_2')
tdml_df_with_index_col

emp_name,emp_sage,emp_id,marks,index_label
A3,300,155,94.67,2
A4,400,177,91.0,3
A2,200,144,97.32,1
A1,100,133,99.99,0


In [26]:
# No index will be set to teradataml DataFrame.
tdml_df_with_index_col.index

#### Usecase 5: Save Pandas DataFrame with index to Teradata Vantage table and retain index column in destination table and set name to that column.

In [28]:
copy_to_sql(df=pandas_df,
            table_name='copy_to_pd_table_3',
            index=True,
            index_label='new_column')

In [29]:
tdml_df_with_index_col = DataFrame('copy_to_pd_table_3')
tdml_df_with_index_col

emp_name,emp_sage,emp_id,marks,new_column
A3,300,155,94.67,2
A4,400,177,91.0,3
A2,200,144,97.32,1
A1,100,133,99.99,0


In [30]:
# No index will be set to teradataml DataFrame.
tdml_df_with_index_col.index

#### Usecase 6: Save Pandas DataFrame having index of type MultiIndex to Teradata Vantage table and retain index columns.

In [31]:
indexed_pandas_df = pandas_df.set_index(['emp_id', 'emp_name'], inplace=False)

In [32]:
indexed_pandas_df.index 

MultiIndex([(133, 'A1'),
            (144, 'A2'),
            (155, 'A3'),
            (177, 'A4')],
           names=['emp_id', 'emp_name'])

In [33]:
indexed_pandas_df

,,emp_sage,marks
emp_id,emp_name,,
133,A1,100,99.99
144,A2,200,97.32
155,A3,300,94.67
177,A4,400,91.00


In [34]:
copy_to_sql(df=indexed_pandas_df,
            table_name='copy_to_pd_index',
            index=True)

In [35]:
indexed_tdml_df = DataFrame('copy_to_pd_index')

In [36]:
indexed_tdml_df

emp_sage,marks,emp_id,emp_name
300,94.67,155,A3
400,91.0,177,A4
200,97.32,144,A2
100,99.99,133,A1


In [37]:
# No index will be set to teradataml DataFrame.
indexed_tdml_df.index

#### Usecase 7: Save Pandas DataFrame having index of type MultiIndex to Teradata Vantage table, retain index columns and set names to those columns.

In [38]:
indexed_pandas_df.index

MultiIndex([(133, 'A1'),
            (144, 'A2'),
            (155, 'A3'),
            (177, 'A4')],
           names=['emp_id', 'emp_name'])

In [39]:
indexed_pandas_df

,,emp_sage,marks
emp_id,emp_name,,
133,A1,100,99.99
144,A2,200,97.32
155,A3,300,94.67
177,A4,400,91.00


In [40]:
copy_to_sql(df=indexed_pandas_df,
            table_name='copy_to_pd_multi_index',
            index=True,
            index_label=['index_1', 'index_2'])

In [41]:
multi_indexed_tdml_df = DataFrame('copy_to_pd_multi_index')

In [42]:
multi_indexed_tdml_df

emp_sage,marks,index_1,index_2
300,94.67,155,A3
400,91.0,177,A4
200,97.32,144,A2
100,99.99,133,A1


In [44]:
# No index will be set to teradataml DataFrame.
multi_indexed_tdml_df.index

#### Usecase 8: Save Pandas DataFrame in target schema.

Note:
- User must have create table rights on provided schema.

In [45]:
schema_name = getpass.getpass("Schema name: ")

Schema name:  ········


In [46]:
copy_to_sql(df=pandas_df,
            table_name='copy_to_pd_table_4',
            schema_name=schema_name)

In [47]:
DataFrame(in_schema(schema_name, 'copy_to_pd_table_4'))

emp_name,emp_sage,emp_id,marks
A3,300,155,94.67
A4,400,177,91.0
A2,200,144,97.32
A1,100,133,99.99


<hr>
<b style = 'font-size:16px;font-family:Arial;color:#E37C4D'>2.1.2 copy_to_sql: Source- teradataml DataFrame, Destination- Teradata Vantage</b>

#### Usecase 1: Save teradataml DataFrame to Teradata Vantage by only specifying table name.

##### Note:
- If table already exists, new data is appended to existing table. If table does not exist, `copy_to_sql()` creates table and loads data.

In [48]:
copy_to_sql(df=tdml_df,
            table_name='copy_to_tdml_table_1')

In [49]:
DataFrame('copy_to_tdml_table_1')

id,masters,gpa,stats,programming,admitted
34,yes,3.85,Advanced,Beginner,0
32,yes,3.46,Advanced,Beginner,0
11,no,3.13,Advanced,Advanced,1
30,yes,3.79,Advanced,Novice,0
28,no,3.93,Advanced,Advanced,1
16,no,3.7,Advanced,Advanced,1
9,no,3.82,Advanced,Advanced,1
13,no,4.0,Advanced,Novice,1
15,yes,4.0,Advanced,Advanced,1
17,no,3.83,Advanced,Advanced,1


####  Usecase 2: Save teradataml DataFrame to Teradata Vantage in a fresh table.

In [50]:
copy_to_sql(df=tdml_df,
            table_name='copy_to_tdml_table_1',
            if_exists='replace')

In [51]:
DataFrame('copy_to_tdml_table_1')

id,masters,gpa,stats,programming,admitted
34,yes,3.85,Advanced,Beginner,0
32,yes,3.46,Advanced,Beginner,0
11,no,3.13,Advanced,Advanced,1
30,yes,3.79,Advanced,Novice,0
28,no,3.93,Advanced,Advanced,1
16,no,3.7,Advanced,Advanced,1
9,no,3.82,Advanced,Advanced,1
13,no,4.0,Advanced,Novice,1
15,yes,4.0,Advanced,Advanced,1
17,no,3.83,Advanced,Advanced,1


#### Usecase 3: Save teradataml DataFrame to Teradata Vantage as a SET table. Also mention column to be used as primary index for destination table.

In [52]:
copy_to_sql(df=tdml_df,
            table_name='copy_to_tdml_set_table',
            set_table=True,
            primary_index='id')

In [53]:
set_table_df = DataFrame('copy_to_tdml_set_table')

In [54]:
set_table_df.index

['id']

In [55]:
set_table_df

id,masters,gpa,stats,programming,admitted
34,yes,3.85,Advanced,Beginner,0
32,yes,3.46,Advanced,Beginner,0
11,no,3.13,Advanced,Advanced,1
40,yes,3.95,Novice,Beginner,0
38,yes,2.65,Advanced,Beginner,1
36,no,3.0,Advanced,Novice,0
7,yes,2.33,Novice,Novice,1
26,yes,3.57,Advanced,Advanced,1
19,yes,1.98,Advanced,Advanced,0
13,no,4.0,Advanced,Novice,1


#### Usecase 4: Save teradataml DataFrame in destination schema.

Notes:
- User must have create table rights on provided schema.

In [56]:
schema_name = getpass.getpass("Schema name: ")

Schema name:  ········


In [57]:
copy_to_sql(df=tdml_df,
            table_name='copy_to_tdml_table_2',
            schema_name=schema_name)

In [58]:
DataFrame(in_schema(schema_name, 'copy_to_tdml_table_2'))

id,masters,gpa,stats,programming,admitted
38,yes,2.65,Advanced,Beginner,1
7,yes,2.33,Novice,Novice,1
26,yes,3.57,Advanced,Advanced,1
5,no,3.44,Novice,Novice,0
3,no,3.7,Novice,Beginner,1
22,yes,3.46,Novice,Beginner,0
24,no,1.87,Advanced,Novice,1
36,no,3.0,Advanced,Novice,0
19,yes,1.98,Advanced,Advanced,0
40,yes,3.95,Novice,Beginner,0


#### Usecase 5 : Save teradataml DataFrame having timestamp data into Teradata Vantage as a PTI table.

Notes:
- `primary_time_index_name`, `timecode_column`, `timebucket_duration`, `timezero_date`, `columns_list`, `sequence_column`, `seq_max` are only applicable for time series data.

In [59]:
copy_to_sql(df=tdml_time_df,
            table_name='copy_to_pti_table',
            timecode_column='clicktime',
            primary_time_index_name='pti_name',
            timebucket_duration='HOURS(2)',
            timezero_date="DATE '2009-01-01'",
            columns_list='event',
            sequence_column='adid',
            seq_max=10)

In [61]:
time_tdml_df = DataFrame('copy_to_pti_table')
time_tdml_df

TD_TIMECODE,TD_SEQNO,partition_id,productid,event
2009-03-19 01:18:08.000000,1,1199,1001,click
2009-03-19 16:43:09.000000,1,1199,1001,view
2009-03-19 16:43:09.000000,1,1199,1001,view
2009-02-09 15:17:59.000000,4,1263,1001,view
2009-03-09 21:17:59.000000,2,1199,1001,view
2009-03-13 17:17:59.000000,4,1071,1001,view
2009-03-13 17:17:59.000000,4,1071,1001,view
2009-03-19 01:18:09.000000,1,1199,1001,landing_page
2009-03-09 21:17:59.000000,2,1199,1001,view
2009-03-19 01:18:08.000000,1,1199,1001,click


<hr>
<b style = 'font-size:17px;font-family:Arial;color:#E37C4D'>2.2 fastload</b>

##### Notes:
- Teradata recommends to use `fastload()` API only when number of rows in the source DataFrame is greater than 100,000 to have better performance. To insert lesser rows, please use `copy_to_sql()` for optimized performance.
- All Teradata types are not supported with fastload(). For example, target table having BLOB and CLOB data type columns cannot be loaded.
- If destination table already exists, existing table will be dropped and new table will be created by default. Use `if_exists` argument to alter this behavior.
- Index columns in source pandas DataFrame will not be reflected in destination table by default. Use `index`, `index_label` arguments to alter this behavior.
- Newly created destination table is always a MULTISET table.
- Duplicate rows in the source DataFrame are not loaded, if the destination table is a MULTISET table having primary index.
- Data will be loaded in chunks of default size. Use `batch_size` argument to alter this size.
- Default number of data-transfer sessions (smaller of 8 or the number of AMPs available) are opened during fastload. Use `open_sessions` argument to alter this behavior. For additional information about number of Teradata data-transfer sessions opened during fastload, please refer to: https://pypi.org/project/teradatasql/#FastLoad
- Destination table is created in default database. Use `schema_name` argument to alter this behavior.
- Errors and warnings information is not persisted as tables by default, rather it is available as pandas dataframes in returned dictionary containing keys named as `errors_dataframe` and `warnings_dataframe` respectively. Use `save_errors` as `True` in combination with `err_tbl_name`, `warn_tbl_name`, `err_staging_db`, `err_tbl_1_suffix`, `err_tbl_2_suffix` arguments to alter this bevavior.

<hr>
<b style = 'font-size:16px;font-family:Arial;color:#E37C4D'>2.2.1 fastload: Source - pandas DataFrame, Destination - Teradata Vantage</b>

#### Usecase 1: Save Pandas DataFrame to Teradata Vantage table.

##### Note:
- If table already exists, it will be dropped and new table will be created.

In [63]:
fastload(df=pandas_df,
         table_name='fastload_table')

Processed 4 rows in batch 1.


{'errors_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'warnings_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'errors_table': '',
 'warnings_table': '',
 'ERR_1_table': '',
 'ERR_2_table': ''}

In [64]:
DataFrame('fastload_table')

emp_name,emp_sage,emp_id,marks
A2,200,144,97.32
A4,400,177,91.0
A3,300,155,94.67
A1,100,133,99.99


####  Usecase 2: Save teradataml DataFrame to an existing Teradata Vantage table.

In [65]:
fastload(df=pandas_df,
         table_name='fastload_table',
         if_exists='append')

Processed 4 rows in batch 1.


{'errors_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'warnings_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'errors_table': '',
 'warnings_table': '',
 'ERR_1_table': '',
 'ERR_2_table': ''}

In [66]:
DataFrame('fastload_table')

emp_name,emp_sage,emp_id,marks
A2,200,144,97.32
A1,100,133,99.99
A3,300,155,94.67
A1,100,133,99.99
A3,300,155,94.67
A4,400,177,91.0
A4,400,177,91.0
A2,200,144,97.32


#### Usecase 3: Save Pandas DataFrame to Teradata Vantage table and set primary index.

In [67]:
fastload(df=pandas_df,
         table_name='fastload_table_1',
         primary_index=['emp_id'])

Processed 4 rows in batch 1.


{'errors_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'warnings_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'errors_table': '',
 'warnings_table': '',
 'ERR_1_table': '',
 'ERR_2_table': ''}

In [68]:
fastload_tdml_df = DataFrame('fastload_table_1')
fastload_tdml_df

emp_name,emp_sage,emp_id,marks
A1,100,133,99.99
A2,200,144,97.32
A3,300,155,94.67
A4,400,177,91.0


In [69]:
fastload_tdml_df.index

['emp_id']

#### Usecase 4: Save Pandas DataFrame having index to Teradata Vantage and retain index column in pandas DataFrame in destination table.

Note:
- `index_label` is applicable only when `index` is `True`.

In [70]:
pandas_df

,emp_name,emp_sage,emp_id,marks
0,A1,100,133,99.99
1,A2,200,144,97.32
2,A3,300,155,94.67
3,A4,400,177,91.00


In [71]:
indexed_pandas_df = pandas_df.set_index(['emp_id', 'emp_name'], inplace=False)

In [72]:
indexed_pandas_df.index

MultiIndex([(133, 'A1'),
            (144, 'A2'),
            (155, 'A3'),
            (177, 'A4')],
           names=['emp_id', 'emp_name'])

In [73]:
indexed_pandas_df

,,emp_sage,marks
emp_id,emp_name,,
133,A1,100,99.99
144,A2,200,97.32
155,A3,300,94.67
177,A4,400,91.00


In [74]:
fastload(df=indexed_pandas_df,
         table_name='fastload_table_2',
         index=True,
         index_label=['index1', 'index2'])

Processed 4 rows in batch 1.


{'errors_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'warnings_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'errors_table': '',
 'warnings_table': '',
 'ERR_1_table': '',
 'ERR_2_table': ''}

In [75]:
indexed_fastload_tdml_df = DataFrame('fastload_table_2')
indexed_fastload_tdml_df

emp_sage,marks,index1,index2
200,97.32,144,A2
400,91.0,177,A4
300,94.67,155,A3
100,99.99,133,A1


In [76]:
indexed_fastload_tdml_df.index

#### Usecase 5: Save Pandas DataFrame to Teradata Vantage table by opening desired number of data transfer sessions.

In [77]:
fastload(df=pandas_df,
         table_name='fastload_table_3',
         open_sessions=2)

Processed 4 rows in batch 1.


{'errors_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'warnings_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'errors_table': '',
 'warnings_table': '',
 'ERR_1_table': '',
 'ERR_2_table': ''}

In [78]:
DataFrame('fastload_table_3')

emp_name,emp_sage,emp_id,marks
A1,100,133,99.99
A3,300,155,94.67
A4,400,177,91.0
A2,200,144,97.32


#### Usecase 6: Save a Pandas Dataframe to a table in specified target database `schema_name`.

In [79]:
schema_name = getpass.getpass("Destination Schema name: ")

Destination Schema name:  ········


In [80]:
fastload(df=pandas_df,
         table_name='fastload_in_diff_schema',
         schema_name=schema_name,
         types = {'emp_name': VARCHAR, 'emp_sage':INTEGER,
                  'emp_id': BIGINT, 'marks': DECIMAL})

Processed 4 rows in batch 1.


{'errors_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'warnings_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'errors_table': '',
 'warnings_table': '',
 'ERR_1_table': '',
 'ERR_2_table': ''}

In [81]:
# Verify table in target schema.
DataFrame(in_schema(schema_name, 'fastload_in_diff_schema'))

emp_name,emp_sage,emp_id,marks
A2,200,144,97.320
A4,400,177,91.000
A3,300,155,94.670
A1,100,133,99.990


#### Usecase 7: Save a Pandas Dataframe to a Teradata Vantage table in specified target database `schema_name`. Save errors and warnings to database specified with `err_staging_db`. Save errors to table named as `err_tbl_name` and warnings to `warn_tbl_name`.

Note:
- User must have create table rights on provided schema.

In [82]:
# Create a pandas dataframe having one duplicate and one fualty row.
faulty_data_dict = {"C_ID": [301, 301, 302, 303, 304, 305, 306, 307, 308],
                    "C_timestamp": ['2014-01-06 09:01:25', '2014-01-06 09:01:25',
                                    '2015-01-06 09:01:25.25.122200', '2017-01-06 09:01:25.11111',
                                    '2013-01-06 09:01:25', '2019-03-06 10:15:28',
                                    '2014-01-06 09:01:25.1098', '2014-03-06 10:01:02',
                                    '2014-03-06 10:01:20.0000']}

faulty_pandas_df = pd.DataFrame(faulty_data_dict)

In [83]:
staging_db = getpass.getpass("Staging database: ")

Staging database:  ········


In [84]:
# Fastlaod data in non-default schema "target_db" and save erors and warnings in given tables.
fastload(df=faulty_pandas_df,
         table_name='fastload_with_err_warn_tbl_stag_db',
         if_exists='replace',
         primary_index='C_ID',
         schema_name=schema_name,
         types={'C_ID': INTEGER, 'C_timestamp': TIMESTAMP(6)},
         err_tbl_name='fld_errors',
         warn_tbl_name='fld_warnings',
         err_staging_db=staging_db)

Processed 9 rows in batch 1.


{'errors_dataframe':    batch_no                                      error_message
 0         1   [Session 1320] [Teradata Database] [Error 267...,
 'warnings_dataframe':         batch_no                                      error_message
 0  batch_summary   [Session 1319] [Teradata SQL Driver] [Warning...,
 'errors_table': 'staging_db.fld_errors',
 'warnings_table': 'staging_db.fld_warnings',
 'ERR_1_table': '',
 'ERR_2_table': ''}

In [85]:
# Validate loaded data table.
DataFrame(in_schema(schema_name, "fastload_with_err_warn_tbl_stag_db"))

C_ID,C_timestamp
301,2014-01-06 09:01:25.000000
308,2014-03-06 10:01:20.000000
306,2014-01-06 09:01:25.109800
307,2014-03-06 10:01:02.000000
304,2013-01-06 09:01:25.000000
303,2017-01-06 09:01:25.111110
305,2019-03-06 10:15:28.000000


In [86]:
# Validate error table.
DataFrame(in_schema(staging_db, "fld_errors"))

batch_no,error_message
1,"[Session 1320] [Teradata Database] [Error 2673] FastLoad failed to insert 1 of 9 batched rows. Batched row 3 failed to insert because of Teradata Database error 2673 in ""staging_db"".""fastload_with_err_warn_tbl_stag_db"".""C_timestamp"""


In [87]:
# Validate warning table.
DataFrame(in_schema(staging_db, "fld_warnings"))

batch_no,error_message
batch_summary,"[Session 1319] [Teradata SQL Driver] [Warning 518] Found 1 duplicate or faulty row(s) while ending FastLoad of database table ""staging_db"".""fastload_with_err_warn_tbl_stag_db"": expected a row count of 8, got a row count of 7"


#### Usecase 8: Save a Pandas Dataframe to a table in specified target database `schema_name`. Save errors in ERR_1 and ERR_2 tables having user defined suffixes provided in `err_tbl_1_suffix` and `err_tbl_2_suffix`. Source Pandas dataframe is same as previous usecase 7.

Note:
- User must have create table rights on provided schema.

In [88]:
returned_dict = fastload(df=faulty_pandas_df,
                         table_name='fastload_with_err_warn_tbl_stag_db',
                         schema_name=schema_name,
                         if_exists='append',
                         types={'C_ID': INTEGER, 'C_timestamp': TIMESTAMP(6)},
                         save_errors=True,
                         err_staging_db=staging_db,
                         err_tbl_1_suffix="_user_err_1",
                         err_tbl_2_suffix="_user_err_2")
returned_dict

Processed 9 rows in batch 1.


{'errors_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'warnings_dataframe': Empty DataFrame
 Columns: []
 Index: [],
 'errors_table': '',
 'warnings_table': '',
 'ERR_1_table': 'staging_db.ml__fl_stag_1732120328815718_user_err_1',
 'ERR_2_table': 'staging_db.ml__fl_stag_1732120328815718_user_err_2'}

In [89]:
# Extract run time table names created in staging database.
err_1_table_name = returned_dict['ERR_1_table'].split('.')[1]
err_2_table_name = returned_dict['ERR_2_table'].split('.')[1]

In [90]:
# Validate ERR_1 table.
DataFrame(in_schema(schema_name, err_1_table_name))

ErrorCode,ErrorFieldName,DataParcel
2673,F_C_timestamp,b'12E...'


In [91]:
# Validate ERR_2 table.
DataFrame(in_schema(schema_name, err_2_table_name))

C_ID,C_timestamp


<hr>
<b style = 'font-size:17px;font-family:Arial;color:#E37C4D'>2.3 read_csv</b>

##### Notes:
- Considerations about a CSV file:
   - First line in the CSV must be a header line. The header line lists the column names separated by the field separator (e.g. col1,col2,col3).
   - Default field separator is considered to be `,`. Use `sep` argument to specify different field separator.
   - Default quoting character is considered to be `"`. Use `quotechar` argument to specify different quoting character.
- Fastload protocol is used by default. Use `use_fastload` argument to alter this behavior.
- Existing table is dropped and new table is created by default. Use `if_exists` argument to alter this behavior.
- Destination table is created as a permanent table by default. Use `temporary` argument to alter this behavior.
- Destination table is created in default database. Use `schema_name` argument to alter this behavior.
- `schema_name` will be ignored when `temporary=True`.
- Destination table is created as MULTISET table by default. Use `set_table` argument to alter this behavior.
- `primary_time_index_name`, `timecode_column`, `timebucket_duration`, `timezero_date`, `columns_list`, `sequence_column`, `seq_max` arguments are only applicable for time series data.
- `catch_errors_warnings`, `save_errors`, `open_sessions` arguments are only applicable while `use_fastload` is `True`.
- Considerations while using **fastLoad** protocol with read_csv():
    - Teradata recommends to use Fastload protocol when number of rows to be loaded is at least 100,000 as Fastload opens multiple data transfer connections to the database. 
    - If the destination table is a MULTISET table having primary index, duplicate rows in the csv are not loaded.
    - All Teradata types are not supported. For example, target table having BLOB and CLOB data type columns cannot be loaded.
    - If there are any incorrect rows, i.e. due to constraint violations, data type conversion errors, etc., FastLoad protocol ignores those rows and inserts all valid rows. Rows that failed to get inserted are categorized into errors and warnings by FastLoad protocol and these errors and warnings are collected into pandas Dataframe. User can access this information by using appropriate values for `catch_errors_warnings` and `save_errors` arguments.
        - To access error and warning information only in the form of pandas Dataframe use `catch_errors_warnings=True` and `save_errors=False`.
        - To access error and warning information the form of pandas Dataframe as well as in the permanent tables use `catch_errors_warnings=True` and `save_errors=True`.
    - Default number of data-transfer sessions (smaller of 8 or the number of AMPs available) are opened for FastLoad operation. Use `open_sessions` argument to alter this behavior. For additional information about number of Teradata data-transfer sessions opened during fastload, please refer to: https://pypi.org/project/teradatasql/#FastLoad.



In [92]:
# Helper function to write data in CSV file.
def write_data_into_csv(filename, record):
    """ 
    Helper unction to write data into the csv file. 
    """
    with open(filename, 'w', newline='') as csvFile:
        csvWriter = csv.writer(csvFile, delimiter='\n')
        csvWriter.writerow(record)

In [94]:
# Generate a CSV file to be used in following usecases.
record_with_header = ['id,fname,lname,marks',
                      '101,abc,def,200',
                      '102,lmn,pqr,105',
                      '103,def,xyz,492',
                      '101,abc,def,300']

write_data_into_csv('test_file.csv', record_with_header)

<hr>
<b style = 'font-size:16px;font-family:Arial;color:#E37C4D'>2.3.1 read_csv: Source - CSV file, Destination - Teradata Vantage</b>

#### Usecase 1: Save data from CSV file into Teradata Vantage table without fastload.

##### Note:
- If table already exists, it will be dropped and new table will be created.

In [95]:
read_csv(filepath='test_file.csv',
         table_name='read_csv_table',
         use_fastload=False,
         types=OrderedDict(id=BIGINT, fname=VARCHAR, lname=VARCHAR, marks=FLOAT))

id,fname,lname,marks
103,def,xyz,492.0
101,abc,def,300.0
102,lmn,pqr,105.0
101,abc,def,200.0


#### Usecase 2: Save data from CSV file into Teradata Vantage table using fastload protocol.

##### Note:
- If table already exists, it will be dropped and new table will be created.
- `types`should be an ordered dictionary where key is column name and value is corresponding teradatasqlalchemy type.

In [96]:
read_csv(filepath='test_file.csv',
         table_name='read_csv_table_1',
         types=OrderedDict(id=BIGINT, fname=VARCHAR, lname=VARCHAR, marks=FLOAT))

id,fname,lname,marks
101,abc,def,200.0
103,def,xyz,492.0
101,abc,def,300.0
102,lmn,pqr,105.0


#### Usecase 3: Save data from CSV file into an existing Teradata Vantage table.

In [97]:
read_csv(filepath='test_file.csv',
         table_name='read_csv_table_1',
         if_exists='append')

id,fname,lname,marks
102,lmn,pqr,105.0
101,abc,def,200.0
103,def,xyz,492.0
101,abc,def,200.0
103,def,xyz,492.0
101,abc,def,300.0
101,abc,def,300.0
102,lmn,pqr,105.0


#### Usecase 4: Load the data from CSV file into Teradata Vantage table and set primary index provided through `primary_index` argument.

In [98]:
tdml_df_with_pi = read_csv(filepath='test_file.csv',
                           table_name='read_csv_table_pi',
                           primary_index=['fname'],
                           types=OrderedDict(id=BIGINT, fname=VARCHAR, lname=VARCHAR, marks=FLOAT))
tdml_df_with_pi

id,fname,lname,marks
101,abc,def,200.0
101,abc,def,300.0
103,def,xyz,492.0
102,lmn,pqr,105.0


In [99]:
tdml_df_with_pi.index

['fname']

#### Usecase 5: Save data from CSV file into an existing Teradata Vantage table using fastload protocol. Catch all errors and warnings. Also store those in permanent table.

In [100]:
# Generate a csv file having duplicate rows and data with mismatching type.
record_with_header = ['id,fname,lname,marks',
                      '101,abc,def,200',
                      '102,lmn,pqr,105',
                      '103,def,xyz,492',
                      '101,abc,def,300.1']

erroneous_data = record_with_header + ['103,def,xyz,492']

write_data_into_csv('erroneous_data.csv', erroneous_data)

In [101]:
returned_tuple = read_csv(filepath='erroneous_data.csv',
                          table_name='read_csv_table_2',
                          save_errors=True,
                          catch_errors_warnings=True,
                          set_table=True,
                          primary_index='id',
                          types=OrderedDict(id=BIGINT, fname=VARCHAR, lname=VARCHAR, marks=INTEGER))
returned_tuple

(    fname lname  marks
 id                    
 102   lmn   pqr    105
 103   def   xyz    492
 101   abc   def    300
 101   abc   def    200,
 {'errors_dataframe': Empty DataFrame
  Columns: []
  Index: [],
  'warnings_dataframe':                                        error_message
  0   [Session 1339] [Teradata SQL Driver] [Warning...,
  'errors_table': '',
  'warnings_table': 'staging_db.td_fl_read_csv_table_2_warn_267',
  'fastloadcsv_error_tables': ['read_csv_table_2_ERR_1',
   'read_csv_table_2_ERR_2']})

In [102]:
# Verify error and warnings tables.

In [103]:
# TODO: Uncomment after https://teradata-pe.atlassian.net/browse/ELE-6743 is completed.
# DataFrame('my_csv_table_2_ERR_1')

In [104]:
# TODO: Uncomment after https://teradata-pe.atlassian.net/browse/ELE-6743 is completed.
# DataFrame('my_csv_table_2_ERR_2')

In [105]:
warnings_table = returned_tuple[1]['warnings_table'].split('.')[1]
DataFrame(warnings_table)

error_message
"[Session 1339] [Teradata SQL Driver] [Warning 518] Found 1 duplicate or faulty row(s) while ending FastLoad of database table ""staging_db"".""read_csv_table_2"": expected a row count of 5, got a row count of 4"


#### Usecase 6: Save data from CSV file Teradata Vantage table using fastload protocol and open desired number of data trasfer sessions.

##### Note
- Default number of data-transfer sessions (smaller of 8 or the number of AMPs available) are opened for FastLoad operation. For additional information about number of Teradata data-transfer sessions opened during fastload, please refer to: https://pypi.org/project/teradatasql/#FastLoad.

In [106]:
read_csv(filepath='test_file.csv',
         table_name='read_csv_open_sessions',
         open_sessions=2,
         types=OrderedDict(id=BIGINT, fname=VARCHAR, lname=VARCHAR, marks=FLOAT))

id,fname,lname,marks
101,abc,def,200.0
103,def,xyz,492.0
101,abc,def,300.0
102,lmn,pqr,105.0


#### Usecase 7: Save timeseries data from CSV file into a Teradata Vantage table.

In [107]:
# Generate a csv file having timeseries data.
pti_record = ['partition_id,adid,product_id,event,clicktime',
              '1039,1,1001,view,2009-07-29 20:17:59',
              '1039,2,1001,view,2009-04-21 13:17:59',
              '1039,4,1001,landing_page,2009-07-16 11:18:18',
              '1039,5,1001,click,2009-08-19 22:18:02',
              '1071,1,1001,view,2009-06-15 18:17:59']

write_data_into_csv('time_series.csv', pti_record)

In [108]:
read_csv(filepath='time_series.csv',
         table_name='read_csv_pti',
         types=OrderedDict(partition_id=INTEGER, adid=INTEGER, product_id=INTEGER,
                           event=VARCHAR, clicktime=TIMESTAMP),
         primary_time_index_name='pti_name',
         timecode_column='clicktime',
         timebucket_duration='HOURS(2)',
         timezero_date="DATE '2009-01-01'",
         columns_list='event',
         sequence_column='adid',
         seq_max=10)

TD_TIMECODE,TD_SEQNO,partition_id,product_id,event
2009-04-21 13:17:59.000000,2,1039,1001,view
2009-07-16 11:18:18.000000,4,1039,1001,landing_page
2009-06-15 18:17:59.000000,1,1071,1001,view
2009-08-19 22:18:02.000000,5,1039,1001,click
2009-07-29 20:17:59.000000,1,1039,1001,view


<hr>
<b style = 'font-size:17px;font-family:Arial;color:#E37C4D'>2.4 DataFrame.to_sql()</b>

##### Notes:
- Table underlying current teradataml DataFrame and target table can't be same.
- If destination table already exists, operation will fail by default. Use `if_exists` argument to alter this behavior.
- Destination table is created as MULTISET table by default. Use `set_table` argument to alter this behavior.
- Destination table is created as a permanent table by default. Use `temporary` argument to alter this behavior.
- Destination table is created in default database. Use `schema_name` argument to alter this behavior.
- `schema_name` will be ignored when `temporary=True`.
- Primary index of source DataFrame is not preserved in target table.
- `primary_time_index_name`, `timecode_column`, `timebucket_duration`, `timezero_date`, `columns_list`, `sequence_column`, `seq_max` are only applicable for time series data.

#### Usecase 1: Save data from teradataml DataFrame to a different table by only specifying table name.

##### Note:
- If table already exists, operation will fail by default..

In [109]:
filtered_df = tdml_df[(tdml_df.gpa == 4.00)]

In [110]:
filtered_df.to_sql(table_name='to_sql_target')

In [111]:
DataFrame('to_sql_target')

id,masters,gpa,stats,programming,admitted
29,yes,4.0,Novice,Beginner,0
13,no,4.0,Advanced,Novice,1
15,yes,4.0,Advanced,Advanced,1


####  Usecase 2: Append data from teradataml DataFrame to an existing Teradata Vantage table which is not same as underlying table of DataFrame.

In [112]:
tdml_df.to_sql(table_name='to_sql_target',
               if_exists='append')

In [113]:
DataFrame('to_sql_target')

id,masters,gpa,stats,programming,admitted
29,yes,4.0,Novice,Beginner,0
15,yes,4.0,Advanced,Advanced,1
34,yes,3.85,Advanced,Beginner,0
13,no,4.0,Advanced,Novice,1
11,no,3.13,Advanced,Advanced,1
30,yes,3.79,Advanced,Novice,0
32,yes,3.46,Advanced,Beginner,0
17,no,3.83,Advanced,Advanced,1
13,no,4.0,Advanced,Novice,1
15,yes,4.0,Advanced,Advanced,1


#### Usecase 3: Save teradataml DataFrame to Teradata Vantage as a SET table. Also mention column to be used as primary index for destination table.

In [114]:
tdml_df.to_sql(table_name='to_sql_set_table',
               set_table=True,
               primary_index='id')

In [115]:
set_df = DataFrame('to_sql_set_table')
set_df

id,masters,gpa,stats,programming,admitted
34,yes,3.85,Advanced,Beginner,0
32,yes,3.46,Advanced,Beginner,0
11,no,3.13,Advanced,Advanced,1
40,yes,3.95,Novice,Beginner,0
38,yes,2.65,Advanced,Beginner,1
36,no,3.0,Advanced,Novice,0
7,yes,2.33,Novice,Novice,1
26,yes,3.57,Advanced,Advanced,1
19,yes,1.98,Advanced,Advanced,0
13,no,4.0,Advanced,Novice,1


In [116]:
set_df.index

['id']

#### Usecase 4: Save teradataml DataFrame in destination schema.

Notes:
- User must have create table rights on provided schema.

In [117]:
schema_name = getpass.getpass("Schema name: ")

Schema name:  ········


In [118]:
tdml_df.to_sql(table_name='to_sql_target_schema',
               schema_name=schema_name)

In [119]:
DataFrame(in_schema(schema_name, 'to_sql_target_schema'))

id,masters,gpa,stats,programming,admitted
34,yes,3.85,Advanced,Beginner,0
32,yes,3.46,Advanced,Beginner,0
11,no,3.13,Advanced,Advanced,1
30,yes,3.79,Advanced,Novice,0
28,no,3.93,Advanced,Advanced,1
16,no,3.7,Advanced,Advanced,1
9,no,3.82,Advanced,Advanced,1
13,no,4.0,Advanced,Novice,1
15,yes,4.0,Advanced,Advanced,1
17,no,3.83,Advanced,Advanced,1


#### Usecase 5 : Save teradataml DataFrame having timestamp data into Teradata Vantage as a PTI table.

Notes:
- `primary_time_index_name`, `timecode_column`, `timebucket_duration`, `timezero_date`, `columns_list`, `sequence_column`, `seq_max` are only applicable for time series data.

In [120]:
tdml_time_df.to_sql(table_name='to_sql_pti',
                    timecode_column='clicktime',
                    columns_list='event',
                    set_table=True)

In [121]:
pti_df = DataFrame('to_sql_pti')
pti_df

TD_TIMECODE,partition_id,adid,productid,event
2009-07-30 02:20:12.000000,1167,2,1001,page_03
2009-02-09 15:17:59.000000,1263,4,1001,view
2009-03-09 21:17:59.000000,1199,2,1001,view
2009-03-13 17:17:59.000000,1071,4,1001,view
2009-03-19 01:18:08.000000,1199,1,1001,click
2009-03-19 01:18:09.000000,1199,1,1001,landing_page
2009-03-19 01:17:59.000000,1199,1,1001,view
2009-08-24 14:30:05.000000,1039,5,1001,page_03
2009-07-27 06:10:12.000000,1167,2,1001,page_03
2009-07-06 13:00:20.000000,1231,1,1001,page_03


In [122]:
pti_df.index

['event']

## 6. Cleanup

In [123]:
tables_to_drop = ['admissions_train', 'sessionize_table',
                  'copy_to_pd_table', 'copy_to_pd_set_table', 'copy_to_pd_table_1', 'copy_to_pd_table_2', 'copy_to_pd_index', 'copy_to_pd_multi_index',
                  'copy_to_tdml_table_1', 'copy_to_tdml_set_table', 'copy_to_tdml_table_2', 'copy_to_pti_table',
                  'fastload_table', 'fastload_table_1', 'fastload_table_2', 'fastload_table_3', 'fastload_in_diff_schema', 'fastload_with_err_warn_tbl_stag_db',
                  'read_csv_table', 'read_csv_table_1', 'read_csv_table_pi', 'read_csv_table_2', 'read_csv_open_sessions', 'read_csv_pti',
                  'to_sql_target', 'to_sql_set_table', 'to_sql_target_schema', 'to_sql_pti']

In [124]:
# Remove the loaded tables.
for table in tables_to_drop:
    try:
        db_drop_table(table)
    except:
        print("Failed to drop {}.". format(table))
        pass

In [125]:
files_to_delete = ['test_file.csv', 'erroneous_data.csv', 'time_series.csv']

In [126]:
# Remove CSV files.
for csv_file in files_to_delete:
    try:
        os.remove (csv_file)
    except:
        print("Failed to remove {}.". format(csv_file))
        pass

In [127]:
# Removes the current context associated with the Teradata Vantage.
remove_context()

True